In [1]:
import pandas as pd
import datetime
from datetime import date, timedelta
import numpy as np
import time

In [2]:
# read in aqi master file
aqi_df = pd.read_csv('datasets/aqi.csv', index_col = 0)
print(aqi_df.head())
# keep only the columns we absolutely need for efficiency
aqi_df = aqi_df[["STATE_NAME", "COUNTY_CODE", "DATE", "AQI"]]
aqi_df["DATE"] = pd.to_datetime(aqi_df["DATE"], infer_datetime_format=True)
aqi_df.reset_index(drop = True, inplace = True)
aqi_df.head()

C:\Users\Owner\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


  STATE_NAME COUNTY_NAME  STATE_CODE  COUNTY_CODE        DATE  AQI CATEGORY  \
0    Alabama     Calhoun           1           15  1992-01-01   12     Good   
1    Alabama     Calhoun           1           15  1992-01-07   28     Good   
2    Alabama     Calhoun           1           15  1992-01-13    6     Good   
3    Alabama     Calhoun           1           15  1992-01-19   12     Good   
4    Alabama     Calhoun           1           15  1992-01-25   13     Good   

  DEFINING_PARAMETER DEFINING_SITE  NUMBER_OF_SITES_REPORTING  YEAR  
0               PM10   01-015-0001                          1  1992  
1               PM10   01-015-0001                          1  1992  
2               PM10   01-015-0001                          1  1992  
3               PM10   01-015-0001                          1  1992  
4               PM10   01-015-0001                          1  1992  


,STATE_NAME,COUNTY_CODE,DATE,AQI
0,Alabama,15,1992-01-01,12
1,Alabama,15,1992-01-07,28
2,Alabama,15,1992-01-13,6
3,Alabama,15,1992-01-19,12
4,Alabama,15,1992-01-25,13


In [3]:
# read in the fire data master file
fire_df = pd.read_csv('datasets/fires.csv', index_col = 0)
print(fire_df.head())
print(fire_df.columns)
# keep only needed columns for efficiency
fire_df = fire_df[["STATE_NAME", "COUNTY_CODE", "DISCOVERY_DATE", "CONT_DATE", "FIRE_CODE", "FIRE_NAME", "FIRE_SIZE"]]
# turn datetimes into datetime objects
fire_df["DISCOVERY_DATE"] = pd.to_datetime(fire_df["DISCOVERY_DATE"], infer_datetime_format=True)
fire_df["CONT_DATE"] = pd.to_datetime(fire_df["CONT_DATE"], infer_datetime_format=True)
# many fires have no cont date, set to disc date if this occurs
no_cont = pd.isnull(fire_df["CONT_DATE"])
fire_df["CONT_DATE"][no_cont] = fire_df["DISCOVERY_DATE"][no_cont]
fire_df.reset_index(drop = True, inplace = True)
fire_df.head()

C:\Users\Owner\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


  FIRE_CODE       FIRE_NAME  FIRE_YEAR DISCOVERY_DATE  DISCOVERY_DOY  \
0                                                                      
0      BEV0   TONSINA CREEK       2005     2005-04-28            118   
1      BEV0  PETERSON CREEK       2005     2005-05-08            128   
2      BEV0     TWENTY MILE       2005     2005-05-09            129   
3      C63B       MOTORHOME       2007     2007-06-01            152   
4      C63B           POWER       2007     2007-06-09            160   

   DISCOVERY_TIME STAT_CAUSE_DESCR   CONT_DATE  CONT_DOY  CONT_TIME  ...  \
0                                                                    ...   
0          1250.0         Campfire  2005-04-29     119.0     2100.0  ...   
1          1715.0    Miscellaneous  2005-05-10     130.0     1800.0  ...   
2           930.0   Debris Burning  2005-05-09     129.0     1030.0  ...   
3          1415.0    Miscellaneous  2007-06-01     152.0     1640.0  ...   
4          1430.0         Campfire  200

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,STATE_NAME,COUNTY_CODE,DISCOVERY_DATE,CONT_DATE,FIRE_CODE,FIRE_NAME,FIRE_SIZE
0,Alaska,122,2005-04-28,2005-04-29,BEV0,TONSINA CREEK,2.3
1,Alaska,122,2005-05-08,2005-05-10,BEV0,PETERSON CREEK,2.5
2,Alaska,122,2005-05-09,2005-05-09,BEV0,TWENTY MILE,0.1
3,Alaska,122,2007-06-01,2007-06-01,C63B,MOTORHOME,0.1
4,Alaska,122,2007-06-09,2007-06-09,C63B,POWER,0.1


In [4]:
#make a dictionary which has a date as a key and returns indices of fire_df which are fires currently burning on that day
fire_dict = dict()
s = time.time()
print(f"Number Fires: {fire_df.shape[0]}")
for i in range(fire_df.shape[0]):
    state_name = fire_df["STATE_NAME"][i]
    county_code = fire_df["COUNTY_CODE"][i]
    sdate = fire_df["DISCOVERY_DATE"][i]
    edate = fire_df["CONT_DATE"][i]
    delta = edate - sdate
    for j in range(delta.days + 1):
        day_date = sdate + timedelta(days=j)
        key = (state_name, county_code, day_date)
        if key not in fire_dict:
            fire_dict[key] = {i}
        else:
            fire_dict[key].add(i)
    if i % 100000 == 0:
        print(f"At row {i} after {time.time()-s:.2f} sec")

Number Fires: 1201230
At row 0 after 0.01 sec
At row 100000 after 12.91 sec
At row 200000 after 27.43 sec
At row 300000 after 40.25 sec
At row 400000 after 52.58 sec
At row 500000 after 65.55 sec
At row 600000 after 78.04 sec
At row 700000 after 90.77 sec
At row 800000 after 108.38 sec
At row 900000 after 125.22 sec
At row 1000000 after 141.81 sec
At row 1100000 after 157.84 sec
At row 1200000 after 174.26 sec


In [5]:
# make a new dict which just has the count and sum
fire_summary_dict = dict()
s = time.time()
print(len(fire_dict))
for i, key in enumerate(fire_dict):
    fires = fire_dict[key]
    size = 0
    for fire in fires:
        size += fire_df["FIRE_SIZE"][fire]
    fire_summary_dict[key] = (len(fires), size)
    if i%100000 == 0:
        print(f"At row {i} after {time.time() - s:.2f} sec")

1075591
At row 0 after 0.00 sec
At row 100000 after 4.80 sec
At row 200000 after 9.22 sec
At row 300000 after 13.27 sec
At row 400000 after 19.51 sec
At row 500000 after 24.09 sec
At row 600000 after 29.26 sec
At row 700000 after 38.36 sec
At row 800000 after 42.87 sec
At row 900000 after 47.65 sec
At row 1000000 after 53.91 sec


In [6]:
# make dict which is by state, date and does count and sum
fire_summary_state_dict = dict()
for s,c,d in fire_summary_dict:
    count, sum = fire_summary_dict[(s,c,d)]
    if (s,d) in fire_summary_state_dict:
        count_c, sum_c = fire_summary_state_dict[(s,d)]
        fire_summary_state_dict[(s,d)] =  (count_c + count, sum_c + sum)
    else:
        fire_summary_state_dict[(s,d)] =  (count, sum)

In [7]:
# for every row of the aqi dataset, get the count and sum of fires that were burning in that county on that day
# and 1-num_weeks weeks prior, as well as the state on that day and 1-num_weeks weeks prior
num_weeks = 3
new_columns = np.zeros(shape=(num_weeks*4, aqi_df.shape[0]))

s = time.time()
print(f"Number of AQI Entries {aqi_df.shape[0]}")
for i in range(aqi_df.shape[0]):
    date = aqi_df["DATE"][i]
    state_name = aqi_df["STATE_NAME"][i]
    county_code = aqi_df["COUNTY_CODE"][i]
    for j in range(num_weeks):
        offset_date = date - timedelta(days=j)
        key = (state_name, county_code, offset_date)
        if key in fire_summary_dict:
            count, size = fire_summary_dict[key]
            new_columns[j*4,i] = count
            new_columns[j*4+1,i] = size
        key = (state_name, offset_date)
        if key in fire_summary_state_dict:
            count, size = fire_summary_state_dict[key]
            new_columns[j*4+2,i] = count
            new_columns[j*4+3,i] = size
    if i % 100000 == 0:
        print(f"At row {i} after {time.time() - s:.2f} sec")
for j in range(num_weeks):
    aqi_df[f"FIRES_COUNT_COUNTY_{j}_WEEK_PRIOR"] = new_columns[j*4,:]
    aqi_df[f"FIRES_SIZE_COUNTY_{j}_WEEK_PRIOR"] = new_columns[j*4+1,:]
    aqi_df[f"FIRES_COUNT_STATE_{j}_WEEK_PRIOR"] = new_columns[j*4+2,:]
    aqi_df[f"FIRES_SIZE_STATE_{j}_WEEK_PRIOR"] = new_columns[j*4+3,:]

Number of AQI Entries 7243237
At row 0 after 0.06 sec
At row 100000 after 18.55 sec
At row 200000 after 34.06 sec
At row 300000 after 47.07 sec
At row 400000 after 58.74 sec
At row 500000 after 70.52 sec
At row 600000 after 81.44 sec
At row 700000 after 93.65 sec
At row 800000 after 106.99 sec
At row 900000 after 124.87 sec
At row 1000000 after 148.47 sec
At row 1100000 after 169.80 sec
At row 1200000 after 182.28 sec
At row 1300000 after 193.14 sec
At row 1400000 after 206.27 sec
At row 1500000 after 218.46 sec
At row 1600000 after 230.82 sec
At row 1700000 after 242.94 sec
At row 1800000 after 254.03 sec
At row 1900000 after 265.69 sec
At row 2000000 after 276.20 sec
At row 2100000 after 286.99 sec
At row 2200000 after 297.95 sec
At row 2300000 after 308.62 sec
At row 2400000 after 318.57 sec
At row 2500000 after 328.57 sec
At row 2600000 after 338.43 sec
At row 2700000 after 348.19 sec
At row 2800000 after 359.73 sec
At row 2900000 after 369.97 sec
At row 3000000 after 379.23 sec
At

In [8]:
# add previous k AQI reads from the county, fill with oldest value otherwise (note this is just past readings not a set offset like 1 day (had issues with that))
k = 3
grouped_df = aqi_df.set_index('DATE').groupby(['STATE_NAME', 'COUNTY_CODE'])['AQI']
s = time.time()
for offset in range(1,k+1):
    shifted = list(grouped_df.shift(offset))
    aqi_df[f'AQI_{offset}_PRIOR'] = list(shifted)
    col_name = 'AQI' if offset == 1 else f'AQI_{offset-1}_PRIOR'
    aqi_df[f'AQI_{offset}_PRIOR'][pd.isnull(shifted)] = aqi_df[col_name][pd.isnull(shifted)]
    print(f'{offset} after {time.time()-s}')

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


1 after 11.770874977111816
2 after 19.447206735610962
3 after 26.536211013793945


In [13]:
# add next days AQI
offset = set(range(1,8)) | set(range(-1,-8,-1))
print(offset)
s = time.time()
for i in offset:
    next_day_aqi = []
    for k,v in grouped_df:
        for j in range(v.shape[0]):
            next_day = v.index[j] + timedelta(days=i)
            if next_day in v.index:
                next_day_aqi.append(v[next_day])
            else:
                next_day_aqi.append(None)
    print(f'offset {i} done after {time.time()-s}')
    aqi_df[f'AQI_{i}_DAY_OFFSET'] = next_day_aqi

{1, 2, 3, 4, 5, 6, 7, -1, -7, -6, -5, -4, -3, -2}
offset 1 done after 137.78314185142517
offset 2 done after 282.34167408943176
offset 3 done after 432.7793252468109
offset 4 done after 581.7684490680695
offset 5 done after 719.5824837684631
offset 6 done after 865.196448802948
offset 7 done after 1003.1694483757019
offset -1 done after 1145.2144486904144
offset -7 done after 1284.808448791504
offset -6 done after 1424.7214517593384
offset -5 done after 1568.4944524765015
offset -4 done after 1706.270426750183
offset -3 done after 1850.4874279499054
offset -2 done after 1999.0944373607635


In [14]:
#aqi_df = aqi_df.drop('AQI_3402_DAY_OFFSET',axis=1)
aqi_df.head()

,STATE_NAME,COUNTY_CODE,DATE,AQI,FIRES_COUNT_COUNTY_0_WEEK_PRIOR,FIRES_SIZE_COUNTY_0_WEEK_PRIOR,FIRES_COUNT_STATE_0_WEEK_PRIOR,FIRES_SIZE_STATE_0_WEEK_PRIOR,FIRES_COUNT_COUNTY_1_WEEK_PRIOR,FIRES_SIZE_COUNTY_1_WEEK_PRIOR,...,AQI_5_DAY_OFFSET,AQI_6_DAY_OFFSET,AQI_7_DAY_OFFSET,AQI_-1_DAY_OFFSET,AQI_-7_DAY_OFFSET,AQI_-6_DAY_OFFSET,AQI_-5_DAY_OFFSET,AQI_-4_DAY_OFFSET,AQI_-3_DAY_OFFSET,AQI_-2_DAY_OFFSET
0,Alabama,15,1992-01-01,12,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,15,1992-01-07,28,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN
2,Alabama,15,1992-01-13,6,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,32.0,NaN,NaN,NaN,25.0,NaN,NaN,46.0,NaN
3,Alabama,15,1992-01-19,12,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,38.0,NaN,NaN,NaN,46.0,NaN,NaN,36.0,NaN
4,Alabama,15,1992-01-25,13,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,55.0,NaN,NaN,NaN,36.0,NaN,NaN,26.0,NaN


In [15]:
aqi_df.to_csv('datasets/aqi_augmented.csv')